**Zadanie: model klasyfikacji**
Stwórz model klasyfikacji, w oparciu o architekturę sieci neuronowych. Naucz model klasyfikować gatunki kwiatów, wykorzystując zbiór Iris. Pamiętaj, że jest to klasyfikacja wieloklasowa, w związku z tym w ostatniej warstwie sieci powinna być funkcja aktywacji Softmax.

In [52]:
import pandas as pd

In [53]:
url = "https://gist.githubusercontent.com/netj/8836201/raw/iris.csv"

In [54]:
df = pd.read_csv(url)

In [55]:
print(df.head())

   sepal.length  sepal.width  petal.length  petal.width variety
0           5.1          3.5           1.4          0.2  Setosa
1           4.9          3.0           1.4          0.2  Setosa
2           4.7          3.2           1.3          0.2  Setosa
3           4.6          3.1           1.5          0.2  Setosa
4           5.0          3.6           1.4          0.2  Setosa


In [56]:
features = ['sepal.length', 'sepal.width', 'petal.length', 'petal.width'] 
target = 'variety' 

# Podział danych
X, y = df[features], df[target]

print(X.head())  # Cechy
print(y.head())  # Etykiety

   sepal.length  sepal.width  petal.length  petal.width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2
0    Setosa
1    Setosa
2    Setosa
3    Setosa
4    Setosa
Name: variety, dtype: object


In [57]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [58]:
# Sieci neuronowe: One-Hot Encoding jest standardem dla klasyfikacji wieloklasowej, ponieważ funkcja aktywacji softmax w ostatniej warstwie oczekuje wyjścia w tej postaci.
# Inicjalizacja enkodera
onehot_encoder = OneHotEncoder(sparse_output=False)
# Konwersja y na NumPy array i transformacja etykiet
y_encoded = onehot_encoder.fit_transform(np.array(y).reshape(-1, 1))

# one_hot_label = tf.keras.utils.to_categorical(labels[0]) bezpośrednio z tf

In [59]:
# Standaryzacja danych wejściowych
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [60]:
# Podział na zbiór treningowy i testowy z stratify
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y)

In [61]:
# Budowa modelu sieci neuronowej
# softmax jest używana w warstwie wyjściowej, gdy model ma rozwiązywać problem klasyfikacji wieloklasowej
# W warstwach ukrytych lepiej jest stosować funkcje aktywacji takie jak ReLU, tanh lub sigmoid

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(X_train.shape[1],)),  # Warstwa wejściowa
    tf.keras.layers.Dense(16, activation='relu'),  # Pierwsza warstwa ukryta
    tf.keras.layers.Dense(8, activation='relu'),   # Druga warstwa ukryta
    tf.keras.layers.Dense(y_encoded.shape[1], activation='softmax')  # Warstwa wyjściowa
])

In [62]:
# Kompilacja modelu - domyślne parametry
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

""" możliwość ustawiania parametrow
tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0,
    reduction=losses_utils.ReductionV2.AUTO,
    name='categorical_crossentropy'
)

tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
    **kwargs 
"""

" możliwość ustawiania parametrow\ntf.keras.losses.CategoricalCrossentropy(\n    from_logits=False,\n    label_smoothing=0,\n    reduction=losses_utils.ReductionV2.AUTO,\n    name='categorical_crossentropy'\n)\n\ntf.keras.optimizers.Adam(\n    learning_rate=0.001,\n    beta_1=0.9,\n    beta_2=0.999,\n    epsilon=1e-07,\n    amsgrad=False,\n    name='Adam',\n    **kwargs \n"

In [63]:
# wypisujemy wartości warstw modelu
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 16)                  │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 3)                   │              27 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 243 (972.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

In [64]:
# Trenowanie modelu
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - accuracy: 0.0624 - loss: 1.1574 - val_accuracy: 0.1667 - val_loss: 1.0949
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3970 - loss: 1.0748 - val_accuracy: 0.5833 - val_loss: 1.0265
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6540 - loss: 1.0007 - val_accuracy: 0.6250 - val_loss: 0.9671
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6271 - loss: 0.9734 - val_accuracy: 0.6667 - val_loss: 0.9172
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7079 - loss: 0.9210 - val_accuracy: 0.6250 - val_loss: 0.8737
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7082 - loss: 0.8479 - val_accuracy: 0.6250 - val_loss: 0.8356
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7202 - loss: 0.7944 - val_accuracy: 0.5833 - val_loss: 0.8014
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7292 - loss: 0.7590 - val_accuracy: 0.5833 - val_loss

In [65]:
# Ocena modelu na zbiorze testowym
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")



Loss: 0.1730, Accuracy: 0.9667


In [66]:
# Przykładowe przewidywanie
y_pred = model.predict(X_test[:5])
print("Predictions (probabilities):\n", y_pred)
print("Predicted classes:", np.argmax(y_pred, axis=1))
print("True classes:", np.argmax(y_test[:5], axis=1))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
Predictions (probabilities):
 [[9.9661535e-01 2.5442536e-03 8.4045914e-04]
 [5.7610553e-03 1.9657648e-01 7.9766250e-01]
 [3.2582607e-02 9.4710904e-01 2.0308373e-02]
 [2.4515359e-02 9.6043551e-01 1.5049155e-02]
 [9.9855751e-01 9.3277008e-04 5.0973194e-04]]
Predicted classes: [0 2 1 1 0]
True classes: [0 2 1 1 0]
